In [ ]:
from lib import *
import pandas as pd
import matplotlib.pyplot as plt

pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

try:
    dfs = load_data_bq()
except Exception as e:
    dfs = load_data_db()
df_candidaturas = dfs["candidaturas"].sort_values("created_at")
from datetime import timedelta
df_candidaturas['created_at'] = df_candidaturas['created_at'].dt.strftime("%Y-%m-%d")
df_candidaturas["created_at"] = pd.to_datetime(df_candidaturas["created_at"])
df_candidaturas["day_of_week"] = df_candidaturas["created_at"].dt.day_name()
df_candidaturas["month"] = df_candidaturas["created_at"].dt.month_name()
df_candidaturas["day_record_original"] = (df_candidaturas["created_at"] - df_candidaturas["created_at"].min()).dt.days
df_candidaturas["enum_month"] = df_candidaturas["created_at"].dt.month
df_candidaturas["enum_day_of_week"] = df_candidaturas["created_at"].dt.day_of_week
df_candidaturas = df_candidaturas.groupby(["created_at", "driver_id"]).apply(lambda x: x.sample(1)).reset_index(drop=True)
df_candidaturas_original = df_candidaturas.copy()
#1m23s

Erro ao executar a query contract_canceled.sql: 403 POST https://bigquery.googleapis.com/bigquery/v2/projects/formal-purpose-354320/jobs?prettyPrint=false: Access Denied: Project formal-purpose-354320: User does not have bigquery.jobs.create permission in project formal-purpose-354320.

Location: None
Job ID: da2dd23f-1581-4658-9390-e14e52a6dcdc

Erro geral: 403 POST https://bigquery.googleapis.com/bigquery/v2/projects/formal-purpose-354320/jobs?prettyPrint=false: Access Denied: Project formal-purpose-354320: User does not have bigquery.jobs.create permission in project formal-purpose-354320.

Location: None
Job ID: da2dd23f-1581-4658-9390-e14e52a6dcdc

Conexão estabelecida com sucesso!


In [ ]:
# FIM
# FIM
# FIM
# FIM
# FIM
# FIM
# FIM
# FIM
# FIM
# FIM

In [ ]:
# date_range = pd.date_range(start=df_candidaturas["created_at"].min(), end=pd.to_datetime("2030-12-31"), freq='D')
date_range = pd.date_range(start=df_candidaturas["created_at"].min(), end=pd.to_datetime("2030-03-14"), freq='D')

df_dates = pd.DataFrame(date_range, columns=["date"])
df_dates["date"] = df_dates["date"].dt.strftime("%Y-%m-%d")
df_dates["date"] = pd.to_datetime(df_dates["date"])
df_dates["month"] = df_dates["date"].dt.month_name()
df_dates["day_of_week"] = df_dates["date"].dt.day_name()
df_dates["day_record_prediction"] = (df_dates["date"] - df_candidaturas["created_at"].min()).dt.days
df_dates["enum_month_prediction"] = df_dates["date"].dt.month
df_dates["enum_day_of_week_prediction"] = df_dates["date"].dt.day_of_week
df_dates = df_dates.merge(df_candidaturas, left_on=["month", "day_of_week"], right_on=["month", "day_of_week"], how="left")
df_dates["count_by_day"] = df_dates.groupby("date")["date"].transform("count")

#2.9s

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

# Preparar os dados
df = df_candidaturas.copy()
df = df.groupby(["day_record_original", "enum_month", "enum_day_of_week"])["created_at"].count().reset_index()

# Selecionar features (entradas) e alvo (target)
features = ["day_record_original", "enum_month", "enum_day_of_week"]
X = df[features].values
y = df["created_at"].values

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from tensorflow.keras import layers, models

from tensorflow.keras import layers, models

# Modelo melhorado com Batch Normalization
model = models.Sequential()

# Camada de entrada com mais neurônios, BatchNormalization e ativação 'relu'
model.add(layers.Dense(32, input_shape=(X_train.shape[1],)))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.2))

# Camada oculta intermediária com BatchNormalization
model.add(layers.Dense(16))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.2))


# Camada oculta intermediária com BatchNormalization
model.add(layers.Dense(8))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.2))

# Camada de saída com ativação linear (padrão para regressão)
model.add(layers.Dense(1, activation='relu'))

# Compilar o modelo
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Treinar o modelo e salvar o histórico
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=1000,
    batch_size=64,
    verbose=1
)

# Avaliar o modelo no conjunto de teste
loss, mae = model.evaluate(X_test, y_test, verbose=0)
# print(f"Test Loss (MSE): {loss:.4f} - Test MAE: {mae:.4f}")

# Fazer previsões
y_pred = model.predict(X_test)
# print("Previsões:", y_pred.flatten())

# Plotar a curva de convergência do loss
plt.figure(figsize=(10,5))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Curva de Convergência do Loss')
plt.xlabel('Épocas')
plt.ylabel('Loss (MSE)')
plt.legend()
plt.grid(True, linestyle="--", alpha=0.7)
plt.show()


In [ ]:
df = df_candidaturas.copy()
df = df.groupby(["day_record_original", "enum_month", "enum_day_of_week"])["created_at"].count().reset_index()
features = ["day_record_original", "enum_month", "enum_day_of_week"]
X = df[features].values
# scaler = MinMaxScaler()
# X_scaled = scaler.fit_transform(X)
y_pred = model.predict(X)
df["pred"] = y_pred
plt.scatter(df["created_at"], df["pred"])

In [ ]:
df = df_candidaturas.copy()
df = df.groupby(["day_record_original", "enum_month", "enum_day_of_week"])["created_at"].count().reset_index()
df["day_record_original"]  = df["day_record_original"]
features = ["day_record_original", "enum_month", "enum_day_of_week"]
X = df[features].values
y_pred = model.predict(X)
df["pred"] = y_pred


plt.figure(figsize=(20, 10))
plt.scatter(df["day_record_original"], df["created_at"], color="blue", label="Dados originais")
plt.scatter(df["day_record_original"], df["pred"], color="red", label="Modelo ajustado")
plt.title("Ajuste do modelo aos dados reais")


In [ ]:
df = df_dates.copy()
df = df.sample(50000)
features = ["day_record_prediction", "enum_month", "enum_day_of_week"]
X = df[features].values
y_pred = model.predict(X)

df_dates = df.copy()
df_dates["num_of_contracts_predict"] = y_pred
df_dates["weight"] =  df_dates["count_by_day"] / df_dates["count_by_day"].mean() * df_dates["num_of_contracts_predict"] 
df_dates["weight"] = df_dates["weight"] / df_dates.groupby("day_record_prediction")["num_of_contracts_predict"].transform("count")
#7,8s

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


df = df_candidaturas.copy()
df = df.groupby("day_record_original")["created_at"].count().reset_index()

plt.figure(figsize=(20, 10))

plt.scatter(df["day_record_original"], df["created_at"], color="blue", marker="o")
plt.xlabel("Day Record Prediction")
plt.ylabel("Weight")
plt.title("Crossplot: Day Record Prediction vs Weight")
plt.grid(True, linestyle="--", alpha=0.7)


df = df_dates.copy()
df = df.groupby("day_record_prediction")["weight"].sum().reset_index()




plt.scatter(df["day_record_prediction"], df["weight"], color="red", marker="o")
plt.xlabel("Day Record Prediction")
plt.ylabel("Weight")
plt.title("Crossplot: Day Record Prediction vs Weight")
plt.grid(True, linestyle="--", alpha=0.7)
plt.show()


In [ ]:
df_dates.head()

In [ ]:
import pandas as pd
import csv

# Carrega o DataFrame da região e remove duplicatas para garantir unicidade
df_region = pd.read_csv("data/RELATORIO_DTB_BRASIL_DISTRITO.csv", delimiter=";")
df_region = df_region[["Nome_UF", "Nome_Município", "Nome_Mesorregião"]].drop_duplicates()

# Supondo que df_dates já exista, faça uma cópia para os resultados
df_resultados = df_dates.copy()

# Realiza o merge
df_resultados = df_resultados.merge(
    df_region, 
    how="left", 
    left_on=["state", "city"],
    right_on=["Nome_UF", "Nome_Município"]
)

# Remove as colunas desnecessárias e reorganiza as colunas
df_resultados = df_resultados.drop(["Nome_UF", "Nome_Município"], axis=1)

columns = [
    'date', 'month', 'day_of_week', 'city', 'state', 'freights_type',
    'contract_days', 'contract_values', 'vehicle', 'supply', 'cnh_category', 'Nome_Mesorregião', 'weight'
]
df_resultados = df_resultados[columns]

df_resultados.columns = [
    'date', 'month', 'day_of_week', 'city', 'state', 'freights_type',
    'contract_days', 'contract_values', 'vehicle', 'supply', 'cnh_category', 'region', 'weight'
]

df_resultados = df_resultados.sort_values("date")


In [ ]:
display(df_resultados.head())
import json
df = df_resultados
exclude_cols = ['date', 'weight']
json_cols = [col for col in df.columns if col not in exclude_cols]
df['data_json'] = df[json_cols].apply(lambda row: json.dumps(row.to_dict()), axis=1)
df_resultados = df[['date', 'weight', 'data_json']]
display(df_resultados.head())


In [ ]:
df_resultados.to_csv("data/simulacao_candidatos")
from lib_volume import *
insert_into_bigquery(df_resultados, table_id = "simulated_future_candidates")